In [1]:
from pathlib import Path

import torch
from PIL import Image
import pandas as pd

DATASET_PATH = Path('/Users/stple/Documents/Projects_Local/xray_interpreter/data')# / 'images_001'

image_paths = list((DATASET_PATH).rglob("*.png"))

In [28]:
print(f"Found {len(image_paths)} images.")

Found 112120 images.


In [29]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
labels = pd.read_csv(DATASET_PATH / 'Data_Entry_2017.csv')

labels = labels[["Image Index", "Finding Labels", "Patient ID"]]
labels['Labels'] = labels.apply(lambda x: x['Finding Labels'].split('|'), axis=1)

mlb = MultiLabelBinarizer()
labels_matrix = mlb.fit_transform(labels['Labels'])
df_labels = pd.DataFrame(labels_matrix, columns=mlb.classes_, index=labels.index)
labels = pd.concat([labels, df_labels], axis=1)

labels = labels.drop(columns=['Finding Labels', 'Labels', 'No Finding'])

In [45]:
labels.describe()

,Patient ID,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
count,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000,112120.000000
mean,14346.381743,0.103095,0.024759,0.041625,0.020540,0.118775,0.022440,0.015037,0.002025,0.177435,0.051570,0.056466,0.030191,0.012763,0.047289
std,8403.876972,0.304084,0.155391,0.199732,0.141841,0.323524,0.148111,0.121702,0.044950,0.382038,0.221158,0.230821,0.171113,0.112251,0.212256
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7310.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,13993.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,20673.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30805.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [46]:
from sklearn.model_selection import train_test_split

# Get unique patient IDs
patient_ids = labels['Patient ID'].unique()

# Split patient IDs into training and testing sets
train_ids, test_ids = train_test_split(patient_ids, test_size=0.2, random_state=42)

# Filter the labels dataframe based on the split
train_labels = labels[labels['Patient ID'].isin(train_ids)]
test_labels = labels[labels['Patient ID'].isin(test_ids)]

print("Train labels:", train_labels.shape)
print("Test labels:", test_labels.shape)

Train labels: (89826, 16)
Test labels: (22294, 16)


In [48]:
train_labels.describe()

,Patient ID,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
count,89826.000000,89826.000000,89826.000000,89826.000000,89826.000000,89826.000000,89826.000000,89826.000000,89826.000000,89826.000000,89826.000000,89826.000000,89826.000000,89826.000000,89826.000000
mean,14290.180393,0.104112,0.024024,0.041369,0.020584,0.119832,0.022098,0.014717,0.001993,0.175862,0.052390,0.056231,0.030336,0.012658,0.047670
std,8401.086067,0.305408,0.153125,0.199143,0.141989,0.324766,0.147004,0.120420,0.044596,0.380705,0.222814,0.230368,0.171512,0.111793,0.213068
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7244.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,13974.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,20524.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30805.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [49]:
train_labels.to_csv('train.csv', index=False)
test_labels.to_csv('test.csv', index=False)

In [ ]:
split = 'train'

labels = pd.read_csv(f"{split}.csv", index_col=0).drop(columns=['Patient ID'])

# labels.info()

labels_dict = labels.apply(lambda row: row.tolist(), axis=1).to_dict()

tensor_list = []
for key in sorted(labels_dict.keys()):
    tensor_list.append(torch.tensor(labels_dict[key]))

all_labels = torch.stack(tensor_list)

# print(all_labels[0])


<class 'pandas.core.frame.DataFrame'>
Index: 89826 entries, 00000001_000.png to 00030805_000.png
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Atelectasis         89826 non-null  int64
 1   Cardiomegaly        89826 non-null  int64
 2   Consolidation       89826 non-null  int64
 3   Edema               89826 non-null  int64
 4   Effusion            89826 non-null  int64
 5   Emphysema           89826 non-null  int64
 6   Fibrosis            89826 non-null  int64
 7   Hernia              89826 non-null  int64
 8   Infiltration        89826 non-null  int64
 9   Mass                89826 non-null  int64
 10  Nodule              89826 non-null  int64
 11  Pleural_Thickening  89826 non-null  int64
 12  Pneumonia           89826 non-null  int64
 13  Pneumothorax        89826 non-null  int64
dtypes: int64(14)
memory usage: 10.3+ MB


In [9]:
from data import ImageDataset
import torchvision.transforms as T

transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(240),
        T.Lambda(lambda img: img.convert("RGB") if img.mode != "RGB" else img),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # EfficientNet's expectations
    ])

train_dataset = ImageDataset('train', False, transform=transform)
# test_dataset = ImageDataset('test', False)

print(f"Train dataset size: {len(train_dataset)}")
# print(f"Test dataset size: {len(test_dataset)}")

# Load to dataloaders
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

for images, labels in train_loader:
    print(images.shape, labels.shape)
    print(images[0, 0:3, 0:3, 0:3])
    break

Train dataset size: 89826
torch.Size([32, 3, 240, 240]) torch.Size([32, 14])
tensor([[[-2.1179, -2.1008, -2.0665],
         [-2.1179, -2.1008, -2.0665],
         [-2.1179, -2.1008, -2.0665]],

        [[-2.0357, -2.0182, -1.9832],
         [-2.0357, -2.0182, -1.9832],
         [-2.0357, -2.0182, -1.9832]],

        [[-1.8044, -1.7870, -1.7522],
         [-1.8044, -1.7870, -1.7522],
         [-1.8044, -1.7870, -1.7522]]])
